# You can use this as a template to quickly create an Assistant using OpenAI's Assistant API

### Import Packages

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00


In [ ]:
from modules import *
from google.colab import userdata
from openai import OpenAI
OPENAI_KEY = userdata.get('OPENAI_KEY')

client = OpenAI(api_key=OPENAI_KEY) # Initialize OpenAI Client

In [ ]:
!pip show openai | grep Version

Version: 1.13.3


### You can start an assistant from scratch or even use an already made assistant on a thread made from scratch

-> For Assistants -
* Set ```get_premade_assistant = False``` to create a new assistant.
* Set ```get_premade_assistant = True``` if you want to use a previous assistant.

-> For Threads -
* Set ```get_previous_thread = False``` to create a new thread.
* Set ```get_previous_thread = True``` if you want to use a previous thread.


In [ ]:
get_premade_assistant = False
get_previous_thread = False

assistant_id_to_use = "asst_6waRJUR4EfaVRYWLkqetgSuu"
thread_id_to_use = "thread_bBkIkcD7yZW3QtrIYt6GMj5e"

### Get Assistant using assistant id

In [ ]:
if get_premade_assistant:
    assistant = get_assistant(client, assistant_id_to_use) # Retrieve Assistant
    print(assistant.name + " is ready to go!")
else:
    name = "Role Generation"
    description = "Near8 Narrative Designer"
    instructions = "You are a narrative designer who designs unique roles based on Club details and User interests\
        Describe their role incorporating the chosen interests without naming them explicitly\
        Do not use the following words in output: 'fantasy, comedy, nature, time travel, cats, horror, true crime, sports, dogs, pop stars, travel, history, romcom, video games, anime, blockchain, asmr, cottagecore'\
        These roles are defined with a name and an attractive description assigned to a user\
        Make sure to create the name as a character name, a first name and a last name that belongs to a fantasy superhero world\
        Create an image for the role, limit it in one line\
        Use a few emojis in output\
        Provide the output in JSON structure like this {'roleName': '<The name of the role>', 'roleDescription': '<The descritpion of the role>',  'imageDes' : '<The image>'}"
    tools = [
        {type: "code_interpreter"},
        {type: "retrieval"}
    ]
    assistant = create_assistant(client, name, description, instructions) # Create Assistant
    print("New Assistant created with ID: " + assistant.id)

New Assistant created with ID: asst_hNcvSNYwqVhPDBBCko6lnWw6


### Retrieve the previous conversation

In [ ]:
# Retrieve the previous conversation thread

if get_previous_thread:
    thread = get_chat(client, thread_id_to_use)
    print("Chat retrieved with ID: " + thread.id)
    print(thread)
else:
    thread = start_new_chat(client)
    print("New chat created with ID: " + thread.id)

New chat created with ID: thread_dBYpuxy87JS4nQj6HTx8oP8C


### Add new message into thread

In [ ]:
# Message to send to the assistant

content = '"Club Name: Vampires Of Brooklyn , User Interests : Yoga, Cycling, Tech , AI"'

In [ ]:
# Add the message into the thread

new_message = add_message(client, thread, content)
print(new_message)

ThreadMessage(id='msg_XE0XsZJH0sSjQXo2Ku5dTwUv', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='"Club Name: Vampires Of Brooklyn , User Interests : Yoga, Cycling, Tech , AI"'), type='text')], created_at=1709938545, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_dBYpuxy87JS4nQj6HTx8oP8C')


### Run the thread with the new message

In [ ]:
# Run the thread with the assistant with the new message

run_chat(client, thread, assistant)

Run(id='run_fTggnk3CAKcGCRrJVuqUPZAa', assistant_id='asst_hNcvSNYwqVhPDBBCko6lnWw6', cancelled_at=None, completed_at=None, created_at=1709938555, expires_at=1709939155, failed_at=None, file_ids=[], instructions="You are a narrative designer who designs unique roles based on Club details and User interests\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Describe their role incorporating the chosen interests without naming them explicitly\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Do not use the following words in output: 'fantasy, comedy, nature, time travel, cats, horror, true crime, sports, dogs, pop stars, travel, history, romcom, video games, anime, blockchain, asmr, cottagecore'\xa0\xa0\xa0\xa0\xa0\xa0\xa0 These roles are defined with a name and an attractive description assigned to a user\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Make sure to create the name as a character name, a first name and a last name that belongs to a fantasy superhero world\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Create an image for the role, limit it in one line\xa

# Run the below code everytime you need to see the new chats

In [ ]:
# Retrieve the chat history

history = get_messages_in_chat(client, thread)
messages = history.data[::-1]
for i in messages:
    print(i.role.upper() + ": "+ i.content[0].text.value)

USER: "Club Name: Vampires Of Brooklyn , User Interests : Yoga, Cycling, Tech , AI"
ASSISTANT: {
	"roleName": "Aria Sunshadow",
	"roleDescription": "Aria Sunshadow is a brilliant tech engineer by day and a graceful yoga instructor by night. She is also a skilled cyclist who traverses the city streets like a shadow, using her expertise in AI to ensure the safety of her fellow cyclists. Aria protects the community not with fangs, but with her intellect and agile movements.",
	"imageDes" : "Aria Sunshadow: A tech-savvy yoga instructor and cyclist."
}


# Multi Assistant Runs

In [ ]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [ ]:
def threadMessageLog(thread):
  global role_name
  # Show the final results


  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )

  # Save the text of the messages so that they can be printed in reverse order
  messageStore = []

  for message in messages:

      if message.assistant_id == assistantRoleGeneration.id:
          assistantName = "Role: "

          # prompt: set a var role_name that extracts the value from a json that may have the attribute roleName. Check that the string is json and that the attribute exists before setting the var
          try:
              parsed_json = json.loads(message.content[0].text.value)
              if "roleName" in parsed_json:
                  role_name = parsed_json["roleName"]
                  print(f"Role name extracted: {role_name}")
              else:
                  print("JSON does not contain 'roleName' attribute")
          except ValueError as e:
              print(f"Invalid JSON: {e}")


      elif message.assistant_id == assistantPostGeneration.id:
          assistantName = "Post: "

      messageStore.append(assistantName+message.content[0].text.value)

  #To make it more readable print the messages in reversed order

  for message in reversed(messageStore):
      print(message)

In [ ]:
import time
name = "Role Generation"
description = "Near8 Role Designer"
instructions = "You are a narrative designer who designs unique roles based on Club details and User interests\
        Describe their role incorporating the chosen interests without naming them explicitly\
        Do not use the following words in output: 'fantasy, comedy, nature, time travel, cats, horror, true crime, sports, dogs, pop stars, travel, history, romcom, video games, anime, blockchain, asmr, cottagecore'\
        These roles are defined with a name and an attractive description assigned to a user\
        Make sure to create the name as a character name, a first name and a last name that belongs to a fantasy superhero world\
        Create an image for the role, limit it in one line\
        Use a few emojis in output\
        Provide the output in JSON structure like this {'roleName': '<The name of the role>', 'roleDescription': '<The descritpion of the role>',  'imageDes' : '<The image>'}"
tools = [
        {type: "code_interpreter"},
        {type: "retrieval"}
    ]
assistantRoleGeneration = create_assistant(client,name,description,instructions)
print("Role Generation Assistant created with ID: " + assistantRoleGeneration.id)



Role Generation Assistant created with ID: asst_X4C2DE7QfoIJSEWMwOkueHU9


In [ ]:
name = "Post Generation"
description = "Near8 Post Designer"
instructions = "You are a narrative designer who designs post and a fortune cookie message using user input\
    Users input is title/description/scene, ,place,Role name,Club name\
  Make sure the caption is short, tweet-sized one-sentence plot points to flesh out an existing storyline\
  Make sure that fortune cookie message in the format of social post like instagram with a limit of 60 words\
  Assign a catchy name to this post\
  Provide the output in JSON structure like this {'1': '<name>', '2': '<caption>', '3': '<social-post>'}"
tools = [
        {type: "code_interpreter"},
        {type: "retrieval"}
    ]
assistantPostGeneration = create_assistant(client,name,description,instructions)
print("Post Generation Assistant created with ID: " + assistantPostGeneration.id)



Post Generation Assistant created with ID: asst_WATC9kbLQIBcwlNHI3B0G6TB


In [ ]:
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [ ]:

def runAssistant(assistant_id,thread_id):
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )
    print(f'Run Assistant Created {run.id}')
    return run


In [ ]:
# create new thread per new user input

threadDuo = client.beta.threads.create()
show_json(threadDuo)

# prompt: ask the user to input Club Name, and User Interests. Then, save it into a var content like such content = '"Club Name: DJs Of Brooklyn , User Interests : Rowing, Surfing, Tech , AI"'

club_name = input("Enter Club Name: ")
user_interests = input("Enter User Interests: ")
content = '"Club Name: {}, User Interests : {}"'.format(club_name, user_interests)


{'id': 'thread_V4y7mbBK5sGEwcEX3ERnXQie',
 'created_at': 1709952785,
 'metadata': {},
 'object': 'thread'}

Enter Club Name: CookieMonster
Enter User Interests: Maltese puppies, belly rubs, walks


In [ ]:
print(content)
message = client.beta.threads.messages.create(
    thread_id=threadDuo.id,
    role="user",
    content=content,
)

show_json(message)

"Club Name: CookieMonster, User Interests : Maltese puppies, belly rubs, walks"


{'id': 'msg_28giT3KFB5k0mXWm3AIC69v9',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': '"Club Name: CookieMonster, User Interests : Maltese puppies, belly rubs, walks"'},
   'type': 'text'}],
 'created_at': 1709952824,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_V4y7mbBK5sGEwcEX3ERnXQie'}

In [ ]:
# Run the Role Assistant to create a first draft
run = runAssistant(assistantRoleGeneration.id,threadDuo.id)


Run Assistant Created run_h3SquW6xy5Ni25XWzNiID2T9


In [ ]:
run = wait_on_run(run, threadDuo)

In [ ]:
threadMessageLog(threadDuo)

Role name extracted: Luna Silverpaws
Role: "Club Name: CookieMonster, User Interests : Maltese puppies, belly rubs, walks"
Role: {
  "roleName": "Luna Silverpaws",
  "roleDescription": "As Luna Silverpaws, you are a playful and nurturing companion to all. Your keen senses and gentle nature make you the ultimate caretaker of adorable creatures. Known for your love of Maltese puppies, belly rubs, and walks, you bring joy and comfort to those around you.",
  "imageDes" : "A compassionate figure with a fluffy companion by her side, radiating warmth and affection. 🐾"
}


In [ ]:
show_json(run)

{'id': 'run_h3SquW6xy5Ni25XWzNiID2T9',
 'assistant_id': 'asst_X4C2DE7QfoIJSEWMwOkueHU9',
 'cancelled_at': None,
 'completed_at': 1709952833,
 'created_at': 1709952831,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': "You are a narrative designer who designs unique roles based on Club details and User interests\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Describe their role incorporating the chosen interests without naming them explicitly\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Do not use the following words in output: 'fantasy, comedy, nature, time travel, cats, horror, true crime, sports, dogs, pop stars, travel, history, romcom, video games, anime, blockchain, asmr, cottagecore'\xa0\xa0\xa0\xa0\xa0\xa0\xa0 These roles are defined with a name and an attractive description assigned to a user\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Make sure to create the name as a character name, a first name and a last name that belongs to a fantasy superhero world\xa0\xa0\xa0\xa0\xa0\xa0\xa0 Create an image for t

In [ ]:
# prompt: ask the user to input a time hook description, a place.

time_hook = input("Enter a time hook description: ")
place = input("Enter a place: ")



Enter a time hook description: Walking around
Enter a place: South park slope


In [ ]:
# prompt: set var postContent with time hook, place, roleName, and clubName e.g. "python day, online ,Sufer Moonlight, DJs of Brooklyn"

postContent = "{}, {}, {}, {}".format(time_hook, place, role_name, club_name)
print(postContent)



Walking around, South park slope, Luna Silverpaws, CookieMonster


In [ ]:
message = client.beta.threads.messages.create(
    thread_id=threadDuo.id,
    role="user",
    content=postContent,
)

show_json(message)

{'id': 'msg_rb3ocqnFTZgXaJF736SIpj3k',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'Walking around, South park slope, Luna Silverpaws, CookieMonster'},
   'type': 'text'}],
 'created_at': 1709953429,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_V4y7mbBK5sGEwcEX3ERnXQie'}

In [ ]:
# Run the Post Assistant
run = runAssistant(assistantPostGeneration.id,threadDuo.id)


Run Assistant Created run_aNsh7kRXTQjXRVLbQ5raBIdN


In [ ]:
run = wait_on_run(run, threadDuo)

In [ ]:

# Show the final results

messages = client.beta.threads.messages.list(
  thread_id=threadDuo.id
)


# Save the text of the messages so that they can be printed in reverse order
messageStore = []

for message in messages:

    if message.assistant_id == assistantRoleGeneration.id:
        assistantName = "Role: "
    elif message.assistant_id == assistantPostGeneration.id:
        assistantName = "Post: "

    messageStore.append(assistantName+message.content[0].text.value)

#To make it more readable print the messages in reversed order

for message in reversed(messageStore):
    print(message)

Role: "Club Name: CookieMonster, User Interests : Maltese puppies, belly rubs, walks"
Role: {
  "roleName": "Luna Silverpaws",
  "roleDescription": "As Luna Silverpaws, you are a playful and nurturing companion to all. Your keen senses and gentle nature make you the ultimate caretaker of adorable creatures. Known for your love of Maltese puppies, belly rubs, and walks, you bring joy and comfort to those around you.",
  "imageDes" : "A compassionate figure with a fluffy companion by her side, radiating warmth and affection. 🐾"
}
Post: Walking around, South park slope, Luna Silverpaws, CookieMonster
Post: {
  "1": "Paws and Cookies",
  "2": "Luna Silverpaws takes a stroll through South Park Slope with the CookieMonster club, spreading joy and warmth wherever she goes.",
  "3": "🐾 Spreading pawsitivity and cookies in South Park Slope with the CookieMonster club! 🍪 #LunaSilverpaws #CookieMonster #SpreadJoy #SouthParkSlope"
}
